op_2024_11_25-27

In [37]:
import pandas as pd
#display all df rows
pd.set_option('display.max_rows', None)
import numpy as np
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

In [15]:
ops_ticker_list = ['ESTC', 'SOFI', 'PLTR'] 
# just_take_gains_list = ['TEVA']
# take_gains_specific_shared_list = ['INTC']

Stock Operations: Monday 11/25 -> Wednesday 11/27 (Week of U.S. Holiday Thanksgiving)
$SOFI, buy_price={$7.50} -SELL 2500sh of 7500sh  @ Key SELL levels between Monday OPEN and Wednesday CLOSE, TP=~ $14.00sh, TR>= $15,000  
->SELL blocks=250sh
$PLTR, buy_price={$7.24} -SELL 500sh of 3000sh @ Key SELL levels between Monday OPEN and Wednesday CLOSE; TP=~ $60.00sh, TR>= $30,000
->SELL block
$ESTC, buy_price={$113.50} -SELL (ALL) 3000sh @ Key SELL levels between Monday OPEN and Wednesday CLOSE; TP=~ $118.18-$120.00sh, TR>= $45,000
->SELL blocks=250sh

MAs (what time peri)
VWAP


Act

In [ ]:
# List of tickers
ops_ticker_list = ['ESTC', 'SOFI', 'PLTR']

# Define the time period
period = '5y'

# Fetch historical price data
hp_data = yf.download(ops_ticker_list, period=period, auto_adjust=True)

# Swap the levels of the MultiIndex columns
hp_df = hp_data.copy()
hp_df.columns = hp_df.columns.swaplevel(0, 1)

# Sort the columns by ticker
hp_df = hp_df.sort_index(axis=1, level=0)

In [23]:
# get datetime today
todays_date = pd.to_datetime('today').normalize()
# strip time from datetime
todays_date = todays_date.strftime('%Y-%m-%d')
# print date
print(todays_date)

2024-11-24


In [34]:
# save to csv
hp_df.to_csv('stock_data.csv')

In [27]:
# Export the DataFrame with MultiIndex columns to Excel
hp_df.to_excel(f'historical_{todays_date}_prices_multiindex.xlsx')


In [35]:
# print(hp_df)

In [36]:
# display(hp_df)

In [46]:
class AdvancedPortfolioTradeSheet:
    def __init__(self, portfolio_config):
        """
        Initialize trading system with portfolio configuration
        """
        self.portfolio = portfolio_config
        self.trade_sheets = {}
        
    def load_data(self, file_path):
        """
        Load and process historical price data from Excel
        """
        # Read the Excel file with multiindex columns
        df = pd.read_excel(file_path, header=[0, 1], index_col=0)
        
        # Process the data for each ticker
        self.price_data = {}
        tickers = list(set(col[0] for col in df.columns if col[0] in ['SOFI', 'PLTR', 'ESTC']))
        
        for ticker in tickers:
            ticker_data = pd.DataFrame()
            ticker_data['Close'] = df[(ticker, 'Close')]
            ticker_data['High'] = df[(ticker, 'High')]
            ticker_data['Low'] = df[(ticker, 'Low')]
            ticker_data['Open'] = df[(ticker, 'Open')]
            ticker_data['Volume'] = df[(ticker, 'Volume')]
            
            # Convert index to datetime if needed
            if not isinstance(ticker_data.index, pd.DatetimeIndex):
                ticker_data.index = pd.to_datetime(ticker_data.index)
            
            self.price_data[ticker] = ticker_data.dropna()
            
        return self.price_data

    def calculate_technical_levels(self, data):
        """
        Calculate comprehensive technical indicators
        """
        df = data.copy()
        
        # Moving Averages
        ma_periods = [5, 10, 20, 50]
        for period in ma_periods:
            df[f'SMA_{period}'] = df['Close'].rolling(window=period).mean()
        
        # Exponential Moving Averages
        df['EMA_9'] = df['Close'].ewm(span=9, adjust=False).mean()
        df['EMA_21'] = df['Close'].ewm(span=21, adjust=False).mean()
        
        # VWAP calculation
        df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3
        df['VWAP'] = (df['Typical_Price'] * df['Volume']).cumsum() / df['Volume'].cumsum()
        
        # Bollinger Bands
        df['BB_Middle'] = df['Close'].rolling(window=20).mean()
        df['BB_Std'] = df['Close'].rolling(window=20).std()
        df['BB_Upper'] = df['BB_Middle'] + (df['BB_Std'] * 2)
        df['BB_Lower'] = df['BB_Middle'] - (df['BB_Std'] * 2)
        
        # RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # Recent support/resistance levels
        recent_data = df.tail(20)
        resistance_levels = recent_data['High'].nlargest(3)
        support_levels = recent_data['Low'].nsmallest(3)
        
        # Volume analysis
        df['Volume_MA_5'] = df['Volume'].rolling(window=5).mean()
        recent_volume = df['Volume'].tail(5).mean()
        
        return {
            'current_price': df['Close'].iloc[-1],
            'vwap': df['VWAP'].iloc[-1],
            'sma_5': df['SMA_5'].iloc[-1],
            'sma_10': df['SMA_10'].iloc[-1],
            'sma_20': df['SMA_20'].iloc[-1],
            'sma_50': df['SMA_50'].iloc[-1],
            'ema_9': df['EMA_9'].iloc[-1],
            'ema_21': df['EMA_21'].iloc[-1],
            'bb_upper': df['BB_Upper'].iloc[-1],
            'bb_middle': df['BB_Middle'].iloc[-1],
            'bb_lower': df['BB_Lower'].iloc[-1],
            'rsi': df['RSI'].iloc[-1],
            'resistance_levels': resistance_levels.values,
            'support_levels': support_levels.values,
            'avg_volume': recent_volume,
            'volume_ma5': df['Volume_MA_5'].iloc[-1],
            'volatility': df['Close'].pct_change().std() * np.sqrt(252)
        }

    def generate_execution_levels(self, technical_data, target_price, entry_price):
        """
        Generate optimized execution levels based on technical analysis
        """
        current_price = technical_data['current_price']
        
        # Calculate price ranges
        price_range = target_price - current_price
        daily_volatility = technical_data['volatility'] / np.sqrt(252)
        
        # Define execution levels with volatility adjustment
        levels = {
            'Aggressive': min(current_price * (1 + daily_volatility), 
                            technical_data['bb_upper']),
            'Target': target_price,
            'Base': max(technical_data['vwap'], 
                       technical_data['sma_20']),
            'Defensive': max(technical_data['bb_lower'],
                           entry_price * 1.05),
            'Stop': max(entry_price * 1.02,
                       technical_data['support_levels'][0])
        }
        
        return levels

    def create_trade_sheet(self, ticker_config, price_data):
        """
        Generate comprehensive trade sheet for a single ticker
        """
        technical_data = self.calculate_technical_levels(price_data)
        execution_levels = self.generate_execution_levels(
            technical_data, 
            ticker_config['target_price'],
            ticker_config['entry_price']
        )
        
        # Calculate optimal block distribution
        total_blocks = ticker_config['shares_to_sell'] // ticker_config['block_size']
        
        execution_schedule = {
            'Monday': {
                'Morning (9:30-11:00)': {
                    'Shares': ticker_config['block_size'] * 2,
                    'Primary Level': execution_levels['Aggressive'],
                    'Secondary Level': execution_levels['Base'],
                    'Stop Level': execution_levels['Defensive']
                },
                'Midday (11:00-14:00)': {
                    'Shares': ticker_config['block_size'] * 2,
                    'Primary Level': execution_levels['Target'],
                    'Secondary Level': execution_levels['Base'],
                    'Stop Level': execution_levels['Defensive']
                },
                'Afternoon (14:00-16:00)': {
                    'Shares': ticker_config['block_size'] * 2,
                    'Primary Level': execution_levels['Base'],
                    'Secondary Level': execution_levels['Defensive'],
                    'Stop Level': execution_levels['Stop']
                }
            },
            'Tuesday': {
                'Morning (9:30-11:00)': {
                    'Shares': ticker_config['block_size'] * 2,
                    'Primary Level': execution_levels['Aggressive'],
                    'Secondary Level': execution_levels['Base'],
                    'Stop Level': execution_levels['Defensive']
                },
                'Midday (11:00-14:00)': {
                    'Shares': ticker_config['block_size'],
                    'Primary Level': execution_levels['Target'],
                    'Secondary Level': execution_levels['Base'],
                    'Stop Level': execution_levels['Defensive']
                },
                'Afternoon (14:00-16:00)': {
                    'Shares': ticker_config['block_size'],
                    'Primary Level': execution_levels['Base'],
                    'Secondary Level': execution_levels['Defensive'],
                    'Stop Level': execution_levels['Stop']
                }
            },
            'Wednesday': {
                'Morning (9:30-11:00)': {
                    'Shares': 'Remaining',
                    'Primary Level': execution_levels['Base'],
                    'Secondary Level': execution_levels['Defensive'],
                    'Stop Level': execution_levels['Stop']
                }
            }
        }
        
        return {
            'ticker': ticker_config['ticker'],
            'entry_price': ticker_config['entry_price'],
            'current_price': technical_data['current_price'],
            'shares_to_sell': ticker_config['shares_to_sell'],
            'block_size': ticker_config['block_size'],
            'target_price': ticker_config['target_price'],
            'min_revenue': ticker_config['min_revenue'],
            'technical_levels': technical_data,
            'execution_levels': execution_levels,
            'execution_schedule': execution_schedule,
            'expected_revenue': technical_data['current_price'] * ticker_config['shares_to_sell']
        }

    def print_trade_sheet(self, sheet):
        """
        Print formatted trade sheet with color coding
        """
        print(f"\nTRADE SHEET - {sheet['ticker']}")
        print("=" * 70)
        
        # Portfolio Statistics
        print("\nPORTFOLIO STATISTICS:")
        print(f"Entry Price: ${sheet['entry_price']:.2f}")
        print(f"Current Price: ${sheet['current_price']:.2f}")
        print(f"Target Price: ${sheet['target_price']:.2f}")
        print(f"Shares to Sell: {sheet['shares_to_sell']:,} (Block Size: {sheet['block_size']})")
        print(f"Expected Revenue: ${sheet['expected_revenue']:,.2f}")
        print(f"Minimum Revenue Target: ${sheet['min_revenue']:,.2f}")
        
        # Technical Levels
        print("\nTECHNICAL LEVELS:")
        print(f"VWAP: ${sheet['technical_levels']['vwap']:.2f}")
        print(f"RSI: {sheet['technical_levels']['rsi']:.1f}")
        print("\nMoving Averages:")
        print(f"  SMA(5): ${sheet['technical_levels']['sma_5']:.2f}")
        print(f"  SMA(20): ${sheet['technical_levels']['sma_20']:.2f}")
        print(f"  EMA(9): ${sheet['technical_levels']['ema_9']:.2f}")
        
        print("\nBollinger Bands:")
        print(f"  Upper: ${sheet['technical_levels']['bb_upper']:.2f}")
        print(f"  Middle: ${sheet['technical_levels']['bb_middle']:.2f}")
        print(f"  Lower: ${sheet['technical_levels']['bb_lower']:.2f}")
        
        # Support/Resistance
        print("\nKEY LEVELS:")
        print("Resistance Levels:")
        for i, level in enumerate(sheet['technical_levels']['resistance_levels'], 1):
            print(f"  R{i}: ${level:.2f}")
        
        print("Support Levels:")
        for i, level in enumerate(sheet['technical_levels']['support_levels'], 1):
            print(f"  S{i}: ${level:.2f}")
        
        # Execution Schedule
        print("\nEXECUTION SCHEDULE:")
        for day, schedule in sheet['execution_schedule'].items():
            print(f"\n{day}:")
            for time_slot, details in schedule.items():
                print(f"\n  {time_slot}:")
                print(f"    Shares: {details['Shares']}")
                print(f"    Primary Level: ${details['Primary Level']:.2f}")
                print(f"    Secondary Level: ${details['Secondary Level']:.2f}")
                print(f"    Stop Level: ${details['Stop Level']:.2f}")

# Example usage
def main():
    # Portfolio configuration
    portfolio_config = [
        {
            'ticker': 'SOFI',
            'entry_price': 7.50,
            'shares_to_sell': 2500,
            'block_size': 250,
            'target_price': 14.00,
            'min_revenue': 15000
        },
        {
            'ticker': 'PLTR',
            'entry_price': 7.24,
            'shares_to_sell': 500,
            'block_size': 100,
            'target_price': 60.00,
            'min_revenue': 30000
        },
        {
            'ticker': 'ESTC',
            'entry_price': 113.50,
            'shares_to_sell': 3000,
            'block_size': 250,
            'target_price': 120.00,
            'min_revenue': 45000
        }
    ]

    # Initialize trading system
    trading_system = AdvancedPortfolioTradeSheet(portfolio_config)
    
    # Load data
    file_path = 'historical_2024-11-24_prices_multiindex.xlsx'  # Update with your file path
    price_data = trading_system.load_data(file_path)
    
    # Generate trade sheets for each ticker
    for config in portfolio_config:
        ticker = config['ticker']
        if ticker in price_data:
            trade_sheet = trading_system.create_trade_sheet(config, price_data[ticker])
            trading_system.print_trade_sheet(trade_sheet)
        else:
            print(f"No data found for {ticker}")

if __name__ == "__main__":
    main()


TRADE SHEET - SOFI

PORTFOLIO STATISTICS:
Entry Price: $7.50
Current Price: $15.60
Target Price: $14.00
Shares to Sell: 2,500 (Block Size: 250)
Expected Revenue: $39,000.00
Minimum Revenue Target: $15,000.00

TECHNICAL LEVELS:
VWAP: $9.07
RSI: 88.4

Moving Averages:
  SMA(5): $14.73
  SMA(20): $12.81
  EMA(9): $14.27

Bollinger Bands:
  Upper: $15.99
  Middle: $12.81
  Lower: $9.63

KEY LEVELS:
Resistance Levels:
  R1: $15.75
  R2: $15.55
  R3: $14.89
Support Levels:
  S1: $9.77
  S2: $10.46
  S3: $10.63

EXECUTION SCHEDULE:

Monday:

  Morning (9:30-11:00):
    Shares: 500
    Primary Level: $15.99
    Secondary Level: $12.81
    Stop Level: $9.63

  Midday (11:00-14:00):
    Shares: 500
    Primary Level: $14.00
    Secondary Level: $12.81
    Stop Level: $9.63

  Afternoon (14:00-16:00):
    Shares: 500
    Primary Level: $12.81
    Secondary Level: $9.63
    Stop Level: $9.77

Tuesday:

  Morning (9:30-11:00):
    Shares: 500
    Primary Level: $15.99
    Secondary Level: $12.81
 

In [49]:
class ESTCOptimizedStrategy:
    def __init__(self, entry_price=113.50, target_price=120.00, total_shares=3000):
        self.entry_price = entry_price
        self.target_price = target_price
        self.total_shares = total_shares
        self.block_size = 250
        
    def analyze_price_momentum(self, price_data):
        """Analyze recent price momentum and volume patterns"""
        df = price_data.copy()
        
        # Calculate momentum indicators
        df['ROC'] = df['Close'].pct_change(5) * 100  # 5-day Rate of Change
        df['Volume_MA'] = df['Volume'].rolling(5).mean()
        df['Price_MA'] = df['Close'].rolling(5).mean()
        
        recent_momentum = {
            'price_trend': df['ROC'].iloc[-1],
            'volume_trend': df['Volume'].iloc[-1] / df['Volume_MA'].iloc[-1],
            'above_ma': df['Close'].iloc[-1] > df['Price_MA'].iloc[-1]
        }
        
        return recent_momentum

    def generate_execution_strategy(self, current_price, technical_data):
        """Generate optimized execution strategy for ESTC"""
        
        # Calculate key price levels ensuring no losses
        price_levels = {
            'Aggressive_Target': 120.00,  # Main target
            'First_Target': 118.18,      # Initial target from recent high
            'Base_Target': 116.50,       # Intermediate target
            'Safety_Level': 114.00,      # Minimum profit level
            'Must_Hold': 113.75          # Absolute minimum (small profit)
        }
        
        # Create dynamic execution schedule
        execution_schedule = {
            'Monday': {
                'Morning (9:30-11:00)': {
                    'Shares': self.block_size,
                    'Primary Level': price_levels['Aggressive_Target'],
                    'Secondary Level': price_levels['First_Target'],
                    'Stop Level': price_levels['Must_Hold'],
                    'Strategy': 'Hold for strength, look for early momentum'
                },
                'Midday (11:00-14:00)': {
                    'Shares': self.block_size,
                    'Primary Level': price_levels['First_Target'],
                    'Secondary Level': price_levels['Base_Target'],
                    'Stop Level': price_levels['Must_Hold'],
                    'Strategy': 'Monitor volume and price action'
                },
                'Afternoon (14:00-16:00)': {
                    'Shares': self.block_size,
                    'Primary Level': price_levels['First_Target'],
                    'Secondary Level': price_levels['Base_Target'],
                    'Stop Level': price_levels['Must_Hold'],
                    'Strategy': 'Look for late day strength'
                }
            },
            'Tuesday': {
                'Morning (9:30-11:00)': {
                    'Shares': self.block_size,
                    'Primary Level': price_levels['Aggressive_Target'],
                    'Secondary Level': price_levels['First_Target'],
                    'Stop Level': price_levels['Must_Hold'],
                    'Strategy': 'Aggressive if Monday closed strong'
                },
                'Midday (11:00-14:00)': {
                    'Shares': self.block_size,
                    'Primary Level': price_levels['First_Target'],
                    'Secondary Level': price_levels['Base_Target'],
                    'Stop Level': price_levels['Must_Hold'],
                    'Strategy': 'Adjust based on morning performance'
                },
                'Afternoon (14:00-16:00)': {
                    'Shares': self.block_size,
                    'Primary Level': price_levels['Base_Target'],
                    'Secondary Level': price_levels['Safety_Level'],
                    'Stop Level': price_levels['Must_Hold'],
                    'Strategy': 'Prepare for Wednesday if targets not hit'
                }
            },
            'Wednesday': {
                'Morning (9:30-11:00)': {
                    'Shares': 'Remaining',
                    'Primary Level': price_levels['Base_Target'],
                    'Secondary Level': price_levels['Safety_Level'],
                    'Stop Level': price_levels['Must_Hold'],
                    'Strategy': 'Final day execution, watch pre-market momentum'
                }
            }
        }
        
        # Add execution rules and observations
        execution_rules = {
            'Primary Rules': [
                "NO SELLING BELOW $114.00 - Protect profit at all costs",
                "Target $120 aggressively in morning sessions if momentum is strong",
                "Use smaller blocks (250) to test price levels initially",
                "Increase block size if we see strong upward momentum",
                "Monitor pre-market and first 30 minutes for direction"
            ],
            'Key Levels': [
                f"Major Target: ${price_levels['Aggressive_Target']:.2f}",
                f"First Target: ${price_levels['First_Target']:.2f}",
                f"Base Target: ${price_levels['Base_Target']:.2f}",
                f"Safety Level: ${price_levels['Safety_Level']:.2f}"
            ],
            'Volume Triggers': [
                "Increase execution size if volume > 1.5x 5-day average",
                "Reduce block size if volume < 0.7x 5-day average",
                "Watch for volume spikes above $116"
            ]
        }
        
        return {
            'price_levels': price_levels,
            'execution_schedule': execution_schedule,
            'execution_rules': execution_rules,
            'current_status': {
                'Current Price': current_price,
                'Distance to Target': f"{((120.00/current_price - 1) * 100):.1f}%",
                'Shares Remaining': self.total_shares,
                'Average Volume': technical_data['avg_volume']
            }
        }

    def print_strategy(self, strategy):
        """Print formatted strategy with emphasis on key points"""
        print("\n$ESTC LED ZEPPELIN STRATEGY")
        print("=" * 70)
        
        # Current Status
        print("\nCURRENT STATUS:")
        for key, value in strategy['current_status'].items():
            print(f"{key}: {value}")
        
        # Price Levels
        print("\nKEY PRICE LEVELS:")
        for level, price in strategy['price_levels'].items():
            print(f"{level.replace('_', ' ')}: ${price:.2f}")
        
        # Execution Rules
        print("\nESTC NV (Elastic Search $ESTC) FOCUSED PRIMARY EXECUTION WORST EVER RULES:")
        for rule in strategy['execution_rules']['Primary Rules']:
            print(f"* {rule}")
        
        # Schedule
        print("\nDETAILED EXECUTION SCHEDULE:")
        for day, times in strategy['execution_schedule'].items():
            print(f"\n{day}:")
            for time_slot, details in times.items():
                print(f"\n  {time_slot}:")
                print(f"    Shares: {details['Shares']}")
                print(f"    Primary Level: ${details['Primary Level']:.2f}")
                print(f"    Secondary Level: ${details['Secondary Level']:.2f}")
                print(f"    Stop Level: ${details['Stop Level']:.2f}")
                print(f"    Strategy: {details['Strategy']}")

# Example usage
estc_strategy = ESTCOptimizedStrategy()
technical_data = {'avg_volume': 1000000}  # Replace with real data
strategy = estc_strategy.generate_execution_strategy(108.03, technical_data)
estc_strategy.print_strategy(strategy)


$ESTC LED ZEPPELIN STRATEGY

CURRENT STATUS:
Current Price: 108.03
Distance to Target: 11.1%
Shares Remaining: 3000
Average Volume: 1000000

KEY PRICE LEVELS:
Aggressive Target: $120.00
First Target: $118.18
Base Target: $116.50
Safety Level: $114.00
Must Hold: $113.75

ESTC NV (Elastic Search $ESTC) FOCUSED PRIMARY EXECUTION WORST EVER RULES:
* NO SELLING BELOW $114.00 - Protect profit at all costs
* Target $120 aggressively in morning sessions if momentum is strong
* Use smaller blocks (250) to test price levels initially
* Increase block size if we see strong upward momentum
* Monitor pre-market and first 30 minutes for direction

DETAILED EXECUTION SCHEDULE:

Monday:

  Morning (9:30-11:00):
    Shares: 250
    Primary Level: $120.00
    Secondary Level: $118.18
    Stop Level: $113.75
    Strategy: Hold for strength, look for early momentum

  Midday (11:00-14:00):
    Shares: 250
    Primary Level: $118.18
    Secondary Level: $116.50
    Stop Level: $113.75
    Strategy: Monito